In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Activation, Dense, Dropout, Flatten
import random
import re
import unicodedata
import sys

Using TensorFlow backend.


In [2]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
print("words:",words)
classes = data['classes']
print("classes:",classes)
train_x = data['train_x']
train_y = data['train_y']
classes_details = data['classes_details']
print("classes details:",classes_details)

words: ['a', 'abdom', 'about', 'ach', 'achiev', 'act', 'aft', 'ago', 'al', 'allergenswhy', 'almost', 'alon', 'along', 'alot', 'already', 'also', 'although', 'alway', 'am', 'an', 'and', 'angry', 'ankl', 'annoy', 'antibiot', 'any', 'anyth', 'ap', 'appear', 'appetit', 'appl', 'ar', 'area', 'arm', 'armp', 'around', 'as', 'at', 'away', 'back', 'bad', 'bank', 'bar', 'be', 'becam', 'becaus', 'becom', 'been', 'beg', 'begin', 'believ', 'bend', 'between', 'big', 'bit', 'blee', 'blist', 'blo', 'block', 'blood', 'bloody', 'body', 'bre', 'brea', 'breast', 'bright', 'bump', 'bunch', 'burn', 'but', 'by', 'cafeter', 'cal', 'can', 'catch', 'caught', 'challeng', 'chang', 'chest', 'chlamydia', 'chronic', 'cle', 'cloud', 'coat', 'cold', 'com', 'coma', 'commun', 'complet', 'concern', 'condit', 'conformatbl', 'confus', 'congest', 'conscy', 'const', 'constip', 'cont', 'contact', 'continu', 'coordin', 'cough', 'could', 'couldnt', 'cramp', 'cri', 'cry', 'cyc', 'cystit', 'day', 'daysit', 'decreas', 'depress', '

In [3]:
# import urllib.request
# import urllib.response
# import sys
# import os, glob
# import http.client, urllib
# import json
# import http.client, urllib.request, urllib.parse, urllib.error, base64

# # Replace the accessKey string value with your valid access key.
# accessKey = '82e4612615634c398bfd26e7d6327833'
# url = 'westcentralus.api.cognitive.microsoft.com'
# path = '/text/analytics/v2.0/keyPhrases'

# def extract_keywords(body):
#     headers = {'Ocp-Apim-Subscription-Key': accessKey}
#     conn = http.client.HTTPSConnection(url)
#     body_req = {'documents':[{'language': 'en', 'id': 1, 'text': body}]}
#     body_json = json.dumps(body_req)
#     conn.request ("POST", path, body_json, headers)
#     response = conn.getresponse ()
#     string = response.read().decode('utf-8')
#     json_obj = json.loads(string)
#     print("json_obj: ", json_obj)
#     keyphrases_list = ((json_obj['documents'])[0])['keyPhrases']
#     print("keyphrases_list: ", keyphrases_list)
#     return keyphrases_list

In [4]:
## a table structure to hold the different punctuation used
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if unicodedata.category(chr(i)).startswith('P'))

## method to remove punctuations from sentences.
def remove_punctuation(text):
     return text.translate(tbl)

def clean_up_sentence(sentence):
    #strip punctuations, numbers, and words containing numbers
    no_punct = remove_punctuation(sentence)
    no_nums1 = ' '.join(s for s in no_punct.split() if not any(c.isdigit() for c in s))
    no_nums2 = re.sub(r'\d+', '', no_nums1)
#     keywords = extract_keywords(no_nums2)
#     keyword_str = ' '.join(s for s in keywords)
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(no_nums2)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    print("sentence after stemming: ", sentence_words)
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [5]:
p = bow("I have recently had a very runny and stuffy nose. The last few days, my throat has also been very sore and I have also developed a severe cough with lots of sneezing", words)
print (p)
print (classes)

sentence after stemming:  ['i', 'hav', 'rec', 'had', 'a', 'very', 'runny', 'and', 'stuffy', 'nos', 'the', 'last', 'few', 'day', 'my', 'throat', 'has', 'also', 'been', 'very', 'sor', 'and', 'i', 'hav', 'also', 'develop', 'a', 'sev', 'cough', 'with', 'lot', 'of', 'sneez']
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [6]:
# load our saved model
model = load_model('my_model.h5')

In [7]:
# i=0
# for x_t in x_tokenized:
#     prediction = model.predict(np.array([x_t]))
#     predicted_label = labels[np.argmax(prediction[0])]
#     print("File ->", test_files[i], "Predicted label: " + predicted_label)
#     i += 1


ERROR_THRESHOLD = 0.01
def classify(sentence):
    # generate probabilities from the model
    prediction = model.predict(np.array([bow(sentence, words)]))
    #print(results)
    # filter out predictions below a threshold
    top_values = [classes[i] for i in np.argsort(prediction[0])[-3:]]
    top_vals_reversed = top_values[::-1]
    predicted_class = classes[np.argmax(prediction[0])]
    # sort by strength of probability
    print("#1 predicted class: " + predicted_class)
    print("Top Values: ",top_vals_reversed)

def more_response(key):
    if key in ["yes", "of course", "sure", "certainly", "absolutely", "yeah", "yep", "yup", "ya", "yea", "aye"]:
        sentence_new = input("Okay, please tell me any possible symptoms. The more details, the better!")
        return response(sentence_new,0)
    elif key in ["negative", "nope", "no", "nah", "no way", "nay"]:
        return print("Okay, thanks for visiting iDoctor. Enjoy the rest of your day!")
    else:
        more = input("Sorry, I couldn't understand that. Would you like more diagnosis?")
        return more_response(more.strip().lower())

def response(sentence, counter):
    if counter <= 2:
        results = classify(sentence)
    # if we have a classification then find the matching intent tag
        #if results and len(sentence) >= 10:
            # loop as long as there are matches to process
            #print("Here are some likely conditions you may have:\n")
            #for i in range(0,min(3,len(results))):
                #print((results[i])[0] + ": " + classes_details[(results[i])[0]])
           # more = input("Is there anything else you would like me to diagnose?")
            #return more_response(more.strip().lower())
#         else:
#             counter += 1
#             sentence_add = input("Unfortunately, I do not have enough information. Please tell me more!")
#             sentence = sentence + " " + sentence_add
#             print("SENTENCE UP UNTIL THIS POINT: " + sentence)
#             return response(sentence, counter)
    else:
        total_responses = ""
        return print("Unfortunately, I cannot help diagnose your medical condition. Please consult a medical professional such as your family doctor. Enjoy the rest of your day!")

def main():
    print("Hi there, my name is iDoctor. Today I will be helping to diagnose any possible medical conditions you may have. Let's get started!")
    name = input("Firstly, what is your name?")
    sentence = input("Hi " + name + "! Please tell me how you are feeling, and any possible symptoms you may be experiencing. The more details, the better!")
    response(sentence, 0)

In [8]:
classify("A few days ago, I went to my friend's house. I ended up getting sweaty and itchy eyes, and rubbing them too much caused a lot of redness of eyes")

sentence after stemming:  ['a', 'few', 'day', 'ago', 'i', 'went', 'to', 'my', 'friend', 'hous', 'i', 'end', 'up', 'get', 'sweaty', 'and', 'itchy', 'ey', 'and', 'rub', 'them', 'too', 'much', 'caus', 'a', 'lot', 'of', 'red', 'of', 'ey']
#1 predicted class: Conjunctivitis
Top Values:  ['Conjunctivitis', 'E Coli Infection', 'Allergies']


In [9]:
print(words)

['a', 'abdom', 'about', 'ach', 'achiev', 'act', 'aft', 'ago', 'al', 'allergenswhy', 'almost', 'alon', 'along', 'alot', 'already', 'also', 'although', 'alway', 'am', 'an', 'and', 'angry', 'ankl', 'annoy', 'antibiot', 'any', 'anyth', 'ap', 'appear', 'appetit', 'appl', 'ar', 'area', 'arm', 'armp', 'around', 'as', 'at', 'away', 'back', 'bad', 'bank', 'bar', 'be', 'becam', 'becaus', 'becom', 'been', 'beg', 'begin', 'believ', 'bend', 'between', 'big', 'bit', 'blee', 'blist', 'blo', 'block', 'blood', 'bloody', 'body', 'bre', 'brea', 'breast', 'bright', 'bump', 'bunch', 'burn', 'but', 'by', 'cafeter', 'cal', 'can', 'catch', 'caught', 'challeng', 'chang', 'chest', 'chlamydia', 'chronic', 'cle', 'cloud', 'coat', 'cold', 'com', 'coma', 'commun', 'complet', 'concern', 'condit', 'conformatbl', 'confus', 'congest', 'conscy', 'const', 'constip', 'cont', 'contact', 'continu', 'coordin', 'cough', 'could', 'couldnt', 'cramp', 'cri', 'cry', 'cyc', 'cystit', 'day', 'daysit', 'decreas', 'depress', 'develop

In [10]:
import coremltools
coreml_model = coremltools.converters.keras.convert('my_model.h5',class_labels=classes)
print("successfully loaded keras model")

coremltools.utils.save_spec(coreml_model, 'my_model.mlmodel')
print("successfully saved coreML model")

ImportError: No module named 'coremltools'